In [28]:
import pandas as pd
import numpy as np
import joblib
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBClassifier
import warnings
warnings.filterwarnings('ignore')

Load data

In [29]:
# load data
df = pd.read_csv('../Data/engineered_data.csv')
df.head()

,datetime,machineID,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Target,RUL_hours,...,machine_age_hours,hours_since_last,Temp_Difference,Temp_Rate_of_Change,Power,Torque_Speed_Ratio,RPM_Variance,Type_H,Type_L,Type_M
0,2015-01-06 19:23:53,1,L,310.0,320.0,1498,39.9,9,0,16893.706389,...,0,8.000000,10.0,0.0,6259.446213,0.026618,0.000000,False,True,False
1,2015-01-11 02:40:41,1,L,310.0,320.0,1500,40.1,15,0,16790.426389,...,8,103.280000,10.0,0.0,6299.220845,0.026716,1.414214,False,True,False
2,2015-01-15 02:42:01,1,L,310.0,320.0,1500,40.1,21,0,16694.404167,...,16,96.022222,10.0,0.0,6299.220845,0.026716,1.154701,False,True,False
3,2015-02-16 07:20:40,1,L,310.0,320.0,1498,40.2,69,0,15921.760000,...,24,772.644167,10.0,0.0,6306.509718,0.026818,1.154701,False,True,False
4,2015-03-07 22:10:47,1,L,310.0,320.0,1499,40.5,98,0,15450.924722,...,32,470.835278,10.0,0.0,6357.814595,0.027000,1.000000,False,True,False


In [30]:
# Filter valid RUL data
df_rul = df[df['RUL_hours'] >= 0].copy()
print(f"Valid RUL records: {len(df_rul):,}")

Valid RUL records: 78,843


Prepare features

In [31]:
FEATURE_COLUMNS = [
    # Sensor features (5)
    'Air_temperature',
    'Process_temperature', 
    'Rotational_speed',
    'Torque',
    'Tool_wear',
    
    # Engineered features (5)
    'Temp_Difference',
    'Power',
    'Torque_Speed_Ratio',
    'Temp_Rate_of_Change',
    'RPM_Variance',
    
    # Datetime features (3)
    'month',
    'hour',
    'dayofweek',
    
    # Temporal features (2)
    'machine_age_hours',
    'hours_since_last',
    
    # Machine type (3)
    'Type_H',
    'Type_L',
    'Type_M'
]

X = df_rul[FEATURE_COLUMNS].values
y = df_rul['RUL_hours'].values

print(f"Feature matrix: {X.shape}")
print(f"Target vector: {y.shape}")

Feature matrix: (78843, 18)
Target vector: (78843,)


In [32]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

In [33]:
# feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Model time-series

In [34]:
# XGB Regressor
model_xgb = XGBRegressor(
    n_estimators=150,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model_xgb.fit(X_train_scaled, y_train, verbose=False)
print("✓ XGBoost trained!")

# Predict
y_pred_xgb_train = model_xgb.predict(X_train_scaled)
y_pred_xgb_test = model_xgb.predict(X_test_scaled)

# Metrics
xgb_train_mae = mean_absolute_error(y_train, y_pred_xgb_train)
xgb_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_xgb_train))
xgb_train_r2 = r2_score(y_train, y_pred_xgb_train)

xgb_test_mae = mean_absolute_error(y_test, y_pred_xgb_test)
xgb_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb_test))
xgb_test_r2 = r2_score(y_test, y_pred_xgb_test)

print(f"\nXGBoost Performance:")
print(f"   Train MAE: {xgb_train_mae:.0f}h ({xgb_train_mae/24:.0f}d) | R²: {xgb_train_r2:.4f}")
print(f"   Test MAE:  {xgb_test_mae:.0f}h ({xgb_test_mae/24:.0f}d) | R²: {xgb_test_r2:.4f}")

✓ XGBoost trained!

XGBoost Performance:
   Train MAE: 190h (8d) | R²: 0.9914
   Test MAE:  219h (9d) | R²: 0.9842


In [35]:
# Random forest
model_rf = RandomForestRegressor(
    n_estimators=150,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train_scaled, y_train)
print("✓ Random Forest trained!")

# Predict
y_pred_rf_train = model_rf.predict(X_train_scaled)
y_pred_rf_test = model_rf.predict(X_test_scaled)

# Metrics
rf_train_mae = mean_absolute_error(y_train, y_pred_rf_train)
rf_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_rf_train))
rf_train_r2 = r2_score(y_train, y_pred_rf_train)

rf_test_mae = mean_absolute_error(y_test, y_pred_rf_test)
rf_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf_test))
rf_test_r2 = r2_score(y_test, y_pred_rf_test)

print(f"\nRandom Forest Performance:")
print(f"   Train MAE: {rf_train_mae:.0f}h ({rf_train_mae/24:.0f}d) | R²: {rf_train_r2:.4f}")
print(f"   Test MAE:  {rf_test_mae:.0f}h ({rf_test_mae/24:.0f}d) | R²: {rf_test_r2:.4f}")


✓ Random Forest trained!

Random Forest Performance:
   Train MAE: 103h (4d) | R²: 0.9939
   Test MAE:  164h (7d) | R²: 0.9860


In [36]:
# gradient boosting regresor
model_gb = GradientBoostingRegressor(
    n_estimators=150,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.8,
    random_state=42
)

model_gb.fit(X_train_scaled, y_train)
print("✓ Gradient Boosting trained!")

# Predict
y_pred_gb_train = model_gb.predict(X_train_scaled)
y_pred_gb_test = model_gb.predict(X_test_scaled)

# Metrics
gb_train_mae = mean_absolute_error(y_train, y_pred_gb_train)
gb_train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_gb_train))
gb_train_r2 = r2_score(y_train, y_pred_gb_train)

gb_test_mae = mean_absolute_error(y_test, y_pred_gb_test)
gb_test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_gb_test))
gb_test_r2 = r2_score(y_test, y_pred_gb_test)

print(f"\nGradient Boosting Performance:")
print(f"   Train MAE: {gb_train_mae:.0f}h ({gb_train_mae/24:.0f}d) | R²: {gb_train_r2:.4f}")
print(f"   Test MAE:  {gb_test_mae:.0f}h ({gb_test_mae/24:.0f}d) | R²: {gb_test_r2:.4f}")

✓ Gradient Boosting trained!

Gradient Boosting Performance:
   Train MAE: 180h (7d) | R²: 0.9928
   Test MAE:  213h (9d) | R²: 0.9846


In [37]:
# model comparasion
# Create comparison table
comparison = pd.DataFrame({
    'Model': ['XGBoost', 'Random Forest', 'Gradient Boosting'],
    'Train MAE (days)': [xgb_train_mae/24, rf_train_mae/24, gb_train_mae/24],
    'Test MAE (days)': [xgb_test_mae/24, rf_test_mae/24, gb_test_mae/24],
    'Train R²': [xgb_train_r2, rf_train_r2, gb_train_r2],
    'Test R²': [xgb_test_r2, rf_test_r2, gb_test_r2],
    'Overfitting Gap': [
        xgb_train_r2 - xgb_test_r2,
        rf_train_r2 - rf_test_r2,
        gb_train_r2 - gb_test_r2
    ]
})

print("\n" + comparison.to_string(index=False))

# Find best model
best_idx = comparison['Test R²'].idxmax()
best_model_name = comparison.loc[best_idx, 'Model']

print(f"\nBEST MODEL: {best_model_name}")
print(f"  Test MAE: {comparison.loc[best_idx, 'Test MAE (days)']:.1f} days")
print(f"  Test R²: {comparison.loc[best_idx, 'Test R²']:.4f}")

# Select best model
if best_model_name == 'XGBoost':
    best_model = model_xgb
    best_test_mae = xgb_test_mae
    best_test_r2 = xgb_test_r2
elif best_model_name == 'Random Forest':
    best_model = model_rf
    best_test_mae = rf_test_mae
    best_test_r2 = rf_test_r2
else:
    best_model = model_gb
    best_test_mae = gb_test_mae
    best_test_r2 = gb_test_r2


            Model  Train MAE (days)  Test MAE (days)  Train R²  Test R²  Overfitting Gap
          XGBoost          7.907136         9.134699  0.991424 0.984237         0.007186
    Random Forest          4.298389         6.824141  0.993859 0.985991         0.007868
Gradient Boosting          7.499616         8.854809  0.992846 0.984628         0.008218

BEST MODEL: Random Forest
  Test MAE: 6.8 days
  Test R²: 0.9860


Model failure classifier

In [38]:
# Get failure samples
from sklearn.metrics import classification_report, confusion_matrix

# Get failure samples (RUL < 1600 hours)
failure_samples = []
for machine_id in df_rul['machineID'].unique():
    machine_data = df_rul[df_rul['machineID'] == machine_id]
    
    # check if machine has failure
    if df[df['machineID'] == machine_id]['Target'].sum() > 0:
        # get failure type
        machine_failure_type = df[df['machineID'] == machine_id][df['Target'] == 1]['Failure_Type'].iloc[0]
        # Get critical readings
        critical_data = machine_data[machine_data['RUL_hours'] < 1600]
        
        for idx in critical_data.index:
            failure_samples.append({
                'index': idx,
                'failure_type': machine_failure_type
            })

failure_df = pd.DataFrame(failure_samples)

In [39]:
# prepare failure classifier data
X_failure = df_rul.loc[failure_df['index'], FEATURE_COLUMNS].values
y_failure_labels = failure_df['failure_type'].values

# encode labels
label_encoder = LabelEncoder()
y_failure = label_encoder.fit_transform(y_failure_labels)

# split data    
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_failure, y_failure,
    test_size=0.2,
    random_state=42,
    stratify=y_failure
)

# scale features    
X_train_f_scaled = scaler.transform(X_train_f)
X_test_f_scaled = scaler.transform(X_test_f)

# train model    
model_failure = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)
    
model_failure.fit(X_train_f_scaled, y_train_f, verbose=False)


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [40]:
# evaluate
from sklearn.metrics import accuracy_score
y_pred_f = model_failure.predict(X_test_f_scaled)
accuracy = accuracy_score(y_test_f, y_pred_f)

# Predictions
y_pred_f = model_failure.predict(X_test_f_scaled)
    
# Accuracy
accuracy = accuracy_score(y_test_f, y_pred_f)
print(f"\n Overall Accuracy: {accuracy:.2%}")
    
# Classification report
print("\n Classification Report:")
print(classification_report(
    y_test_f, y_pred_f,
    target_names=label_encoder.classes_,
    digits=3
))
    
# Confusion matrix
cm = confusion_matrix(y_test_f, y_pred_f)
print(" Confusion Matrix:")
print(cm) 


 Overall Accuracy: 95.35%

 Classification Report:
                          precision    recall  f1-score   support

Heat Dissipation Failure      0.893     0.948     0.920      2152
      Overstrain Failure      0.997     0.996     0.997      1483
           Power Failure      1.000     1.000     1.000      1505
         Random Failures      0.888     0.782     0.831      1123
       Tool Wear Failure      0.996     0.998     0.997      1611

                accuracy                          0.954      7874
               macro avg      0.955     0.945     0.949      7874
            weighted avg      0.953     0.954     0.953      7874

 Confusion Matrix:
[[2041    0    0  111    0]
 [   0 1477    0    0    6]
 [   0    0 1505    0    0]
 [ 245    0    0  878    0]
 [   0    4    0    0 1607]]


Save model

In [41]:
# save best RUL model
joblib.dump(best_model, '../Model/rul_model.pkl')
print(f"RUL model saved: ({best_model_name})")

# save scaler
joblib.dump(scaler, '../Model/scaler.pkl')
print(f"Scaler saved")

# save failure classifier
if model_failure:
    joblib.dump(model_failure, '../Model/failure_model.pkl')
    joblib.dump(label_encoder, '../Model/label_encoder.pkl')
    print("Failure classifier saved")

# Save feature names
with open('../Model/features.json', 'w') as f:
    json.dump(FEATURE_COLUMNS, f, indent=2)
print("Feature list saved")

# Save metadata
metadata = {
    'training_date': pd.Timestamp.now().isoformat(),
    'best_model': best_model_name,
    'models_compared': ['XGBoost', 'Random Forest', 'Gradient Boosting'],
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'features': len(FEATURE_COLUMNS),
    'performance': {
        'test_mae_hours': float(best_test_mae),
        'test_mae_days': float(best_test_mae / 24),
        'test_r2': float(best_test_r2)
    },
    'comparison': comparison.to_dict('records')
}

with open('../Model/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("Metadata saved")

RUL model saved: (Random Forest)
Scaler saved
Failure classifier saved
Feature list saved
Metadata saved
